# Week 3

# 1. Scrape data from the Wikipedia page

Importing pandas library and installing lxml (at the first run)

In [1]:
import pandas as pd

In [2]:
#pip install lxml

Importing data from wikipedia page, converting into a dataframe and then removing "Not assigned" postal codes

In [3]:
#Import data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url, header=0)[0]

#Drop lines with "Not assigned" Postal Code
df = dfs.loc[dfs['Borough'] != "Not assigned"]

#Reset the index and show the first lines of the dataframe
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Checking the shape of the dataframe

In [4]:
#Shape of the dataframe
df.shape

(103, 3)

# 2. Get the latitude and the longitude coordinates of each neighborhood

The following code is used to download the csv file with the coordinates of each neighborhood (geocoder is not running properly)

In [5]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Framing geospatial data in a dataframe

In [6]:
geospatial_data = pd.read_csv("Geospatial_data.csv")
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Two empty arrays for longitude and latitude data are created.
A for loop is used to scan through the Neighbourhoods list, for each item the Postal code is checked in the geospatial_data frame, the corresponding Latitude and Longitude are added to the Latitude and Longitude array. The arrays are then appended to df dataframe

In [7]:
Latitude = []
Longitude = []
for ind in df.index:
    Latitude.append(geospatial_data[geospatial_data['Postal Code'] == df['Postal Code'][ind]]['Latitude'].values[0])
    Longitude.append(geospatial_data[geospatial_data['Postal Code'] == df['Postal Code'][ind]]['Longitude'].values[0])
df['Latitude'] = Latitude
df['Longitude'] = Longitude

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [8]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 3. Cluster and analyze Toronto

#### Use geopy library to get the latitude and longitude values of Toronto.


In [9]:
#!conda install -c conda-forge geopy --yes # only for the first run
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium

#### Get geographical coordinates of Toronto.

In [10]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Prepare the dataset with the Boroughs containing the word Toronto

In [11]:
toronto_data = df[df['Borough'].str.contains("Toronto")]

#### Create the map centered on Toronto and superimpose the centers of the Neighbourhoods

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto